In [108]:
import pandas as pd
import duckdb
from pathlib import Path

# some DuckDB setup
con = duckdb.connect()
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")
# setup postgres
con.execute("""
INSTALL postgres;
LOAD postgres;
""")

con.execute("ATTACH 'dbname=mast_db user=root password=root host=127.0.0.1' AS db (TYPE POSTGRES, READ_ONLY);")

In [21]:
paths = Path('../metadata/M9/signals/').glob('*.parquet')
for path in paths:
    df = pd.read_parquet(path)
    print(df.columns)
    if len(df) == 0 or 'name' not in df.columns:
        path.unlink()

Index(['dataset_item_uuid', 'description', 'file_name', 'format', 'label',
       'mds_name', 'name', 'pass_', 'rank', 'shape', 'shot', 'shot_id',
       'source', 'status', 'time_index', 'type', 'units', 'group'],
      dtype='object')
Index(['dataset_item_uuid', 'description', 'file_name', 'format', 'label',
       'mds_name', 'name', 'pass_', 'rank', 'shape', 'shot', 'shot_id',
       'source', 'status', 'time_index', 'type', 'units', 'group'],
      dtype='object')
Index(['dataset_item_uuid', 'description', 'file_name', 'format', 'label',
       'mds_name', 'name', 'pass_', 'rank', 'shape', 'shot', 'shot_id',
       'source', 'status', 'time_index', 'type', 'units', 'group'],
      dtype='object')
Index(['dataset_item_uuid', 'description', 'file_name', 'format', 'label',
       'mds_name', 'name', 'pass_', 'rank', 'shape', 'shot', 'shot_id',
       'source', 'status', 'time_index', 'type', 'units', 'group'],
      dtype='object')
Index(['dataset_item_uuid', 'description', 'file_nam

In [19]:
%%time
pd.concat([pd.read_parquet(f) for f in Path('../metadata/M9/signals/').glob('*.parquet')])

CPU times: user 17.7 s, sys: 1.7 s, total: 19.4 s
Wall time: 13.9 s


,dataset_item_uuid,description,file_name,format,label,mds_name,name,pass_,rank,shape,...,right,strobe,taps,top,trigger,vbin,view,width,INTERLACE_MODE,_ARRAY_DIMENSIONS
0,1ea514a5-aca6-5425-ba45-77cdfc09502a,Fast ion Gauge: gas pressure,None,None,Fast Ion Gauge molec.den,\TOP.ANALYSED.AGA:FIG,AGA_FIG,0.0,1,[127000],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c9530c89-068e-5712-988d-743e6de7e299,Unfiltered signal for FIG,None,None,Fast Ion Gauge molec.den,\TOP.ANALYSED.AGA:FIG_RAW,AGA_FIG_RAW,0.0,1,[127000],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7a9ad231-816c-58cc-8912-0eab6a18fca7,Gas output for group 1 (TC5 TC11),None,None,Gas output for group 1,\TOP.ANALYSED.AGA:GROUP1,AGA_GROUP1,0.0,1,[8000],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010ef814-75ac-5aa6-98eb-c6b49d3c51fc,Gas output for group 2 (BC5),None,None,Gas output for group 2,\TOP.ANALYSED.AGA:GROUP2,AGA_GROUP2,0.0,1,[8000],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,77a97a64-ff95-5726-bfc2-2e389a39f378,Gas output for group 3 (HL1 HU6 HU8),None,None,Gas output for group 3,\TOP.ANALYSED.AGA:GROUP3,AGA_GROUP3,0.0,1,[8000],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,None,None,None,None,None,None,None,NaN,1,[9504],...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,[time]
519,None,None,None,None,None,None,None,NaN,1,[9504],...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,[time]
520,None,None,None,None,None,None,None,NaN,1,[9504],...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,[time]
521,cd15a98e-f0d4-5280-b805-eca7d1547c79,RBG 2D multi-colour visible bremsstrahlung camera,rgb030473.ipx,IPX,None,None,RGB,-1.0,1219,"[99, 480, 640]",...,640.0,0.0,1.0,1.0,-0.1,0.0,Sector9U,640.0,NaN,None


In [142]:
df = con.execute("""
    SELECT name, description, shape, url FROM db.public.signals 
    ORDER BY name 
""").df()
import intake

cat = intake.open_catalog('../src/api/static/intake/catalog.yml')
url = df.url.iloc[-1]
cat.bucket(url=url).read()

GroupNotFoundError: group not found at path ''

In [91]:
# pd.read_parquet('s3://mast/test/metadata/M9/30420.parquet', storage_options={'s3': {'anon': True, 'endpoint_url': 'https://s3.echo.stfc.ac.uk'}})
pd.io.common.is_fsspec_url = lambda x: True
pd.read_parquet('filecache::https://s3.echo.stfc.ac.uk/mast/test/metadata/M9/30420.parquet')

,name,shape,description,dimensions
0,ABM_CALIB_SHOT,[1],Shot used for calibration (obsolete),[time]
1,ABM_CHANNEL_STATUS,"[1, 32]","Failed = 0, OK = 1","[time, dim_0]"
2,ABM_CHANNEL_TYPE,"[1, 32]","Channel type (0 = poloidal, 1 = co-tangential,...","[time, dim_0]"
3,ABM_GAIN,"[1, 32]",Gain of pre-amplifiers,"[time, dim_0]"
4,ABM_I-BOL,"[7500, 32]",Incident powers (x - channel),"[time, chord_number]"
...,...,...,...,...
888,RCO,"[750, 364, 512, 3]",Phantom colour camera,"[time, height, width, channel]"
889,RGB,"[99, 480, 640]",RBG 2D multi-colour visible bremsstrahlung camera,"[time, height, width]"
890,RGC,"[186, 480, 640]",RBG 2D multi-colour visible bremsstrahlung camera,"[time, height, width]"
891,RIR,"[2296, 8, 320]",Medium wavelength infrared camera,"[time, height, width]"


In [19]:
con.execute("""
    SELECT name, dataset_item_uuid, description, shape FROM read_parquet('s3://mast/test/metadata/M9/signals/30420.parquet', union_by_name=True) 
    WHERE description LIKE '%plasma%'
    ORDER BY name 
    LIMIT 10
""").df()

: 

: 

In [94]:
import intake
source = intake.open_catalog('../cat.yaml')
expr=""" 
    SELECT name, description, shape FROM csv_source 
    WHERE description LIKE '%plasma%'
    ORDER BY name 
    LIMIT 10
"""
source.duck_source(sql_expr=expr).read()
# source.csv_source.read()

,name,description,shape
0,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
1,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
2,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
3,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
4,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
5,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
6,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
7,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
8,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
9,ADA_DALPHA PEAK RADIUS,Outer plasma radius from linear Dalpha,[500]
